In [ ]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import pdfplumber
from ipywidgets import widgets
from io import BytesIO

In [ ]:
# You need to do this one time on your computer
!ollama pull llama3.2

In [ ]:
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
def get_system_prompt():
    system_prompt = """You are an expert research summarizer. That summarizes the content of the research paper in no more than 700 words. The research summary that you provide should include the following:
    1) Objective/Problem - State the research goal or hypothesis.
    2) Background - Briefly explain the context and significance.
    3) Methods - Summarize the approach or methodology.
    4) Key Findings - Highlight the main results or insights.
    5) Conclusion - Provide the implications or contributions of the study.
    6) Future Directions - Suggest areas for further research or exploration.
    7) Limitations - Highlight constraints or challenges in the study.
    8) Potential Applications - Discuss how the findings can be applied in real-world scenarios.
    Keep all points concise, clear, and focused and generate output in markdown."""
    return system_prompt


def messages_for(user_prompt, system_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
def summarize(user_prompt, system_prompt):
    # Generate messages using the user_prompt
    messages = messages_for(user_prompt, system_prompt)
    try:
        response = openai.chat.completions.create(
            model=MODEL,  # Correct model name
            messages=messages,
            max_tokens = 700 # Pass the generated messages
        )
        # Return the content from the API response correctly
        return response.choices[0].message.content
    except Exception as e:
        # Instead of printing, return an error message that can be displayed
        return f"Error in OpenAI API call: {e}"
        
def extract_text_and_generate_summary(change):
    print("Extracting text from pdf.")
    if upload_widget.value:
        # Extract the first uploaded file
        uploaded_file = list(upload_widget.value)[0]
        pdf_file = uploaded_file['content']

        # Extract text from the PDF
        try:
            with pdfplumber.open(BytesIO(pdf_file)) as pdf:
                extracted_text = "\n".join(page.extract_text() for page in pdf.pages)

            # Generate the user prompt
            user_prompt = (
                f"You are looking at the text from a research paper. Summarize it in no more than 700 words. "
                f"The output should be in markdown.\n\n{extracted_text}"
            )

            # Generate the system prompt
            system_prompt = get_system_prompt()

            # Get the summarized response
            response = summarize(user_prompt, system_prompt)
            
            if response:
                # Use IPython's display method to show markdown below the cell
                display(Markdown(response))
        
        except Exception as e:
            # If there's an error, display it using Markdown
            display(Markdown(f"**Error:** {str(e)}"))

    # Reset the upload widget
    upload_widget.value = ()


In [ ]:
upload_widget = widgets.FileUpload(
    accept='.pdf', 
    multiple=False,
    description='Upload PDF',
    layout=widgets.Layout(width='300px',height = '100px', border='2px dashed #cccccc', padding='10px')
)

In [ ]:
upload_widget.observe(extract_text_and_generate_summary, names='value')

In [ ]:
#response seen in logs
display(upload_widget)